#Running SATIVA - final reference database

Prepare input files.
The alignment has been produced using ReproPhylo in the Sativa_final_align_tree notebook. We'll just cleanup the sequence headers and create a local copy.

In [1]:
pwd

u'/home/working/AgainstRefDB/Curat1/FINAL/Sativa'

In [2]:
ls

692921477605289.99_rbcL@mafftLinsi_aln_clipped.fasta
692921477605289.99_rbcL@mafftLinsi_aln_clipped.fasta.reduced
829381477658321.15_rbcL@mafftLinsi_aln_clipped.fasta
829381477658321.15_rbcL@mafftLinsi_aln_clipped.fasta.reduced
955081477588150.11_rbcL.fasta
957451477652905.28_rbcL.fasta
align_eyorks.pkl
alignment.phy
eyorks_flora_curated.gb
eyorks_flora_final.gb
loci_counts.txt
loci.csv
nr_eyorks.pkl
post_SATIVA_align.pkl
postSATIVA_cleaned.fa
postSATIVA_cleaned.gb
post_SATIVA_final.pkl
raw_eyorks.pkl
RAxML_bootstrap.692921477605289.99_rbcL@mafftLinsi_aln_clipped0
RAxML_bootstrap.829381477658321.15_rbcL@mafftLinsi_aln_clipped0
RAxML_info.692921477605289.99_rbcL@mafftLinsi_aln_clipped0
RAxML_info.829381477658321.15_rbcL@mafftLinsi_aln_clipped0
rbcL_cleanforSATIVA_eyorks.gb
rbcL_clustered_cleaned_eyorks.fa
rbcL_clustered_cleaned_eyorks.gb
rbcL_cropped_eyorks.fa
rbcL_cropped_eyorks.gb
rbcL_eyorks_id-1.uc
rbcL_final_cleaned.gb
rbcL.final_epa.jplace
rbcL.l1out_seq.jplace
rbcL.log
rbcL@mafft

In [3]:
!cat rbcL@mafftLinsi_aln_clipped.phy | sed 's/_f[0-9] / /' > alignment.phy

SATIVA requires a 'taxonomy file', i.e. a text file that links each sequence record to a NCBI taxonomic ID (aka taxid).

We are going to generate this file from a bunch of sequence records downloaded from Genbank.

We have downloaded our reference sequences from Genbank in .gb format, which contains metadata including the taxid.

Let's parse the Genbank file.

In [4]:
from Bio import SeqIO

records = SeqIO.to_dict(SeqIO.parse(open('rbcL_nr_pre_Sativa_eyorks.gb','r'),'genbank'))

Before we proceed, there are some records I have manually identified that are still in this Genbank file but either will cause problems downstream, or are absent from the alignment (most of these are sequences that were removed at the clipping stage). Let's remove them now.

In [5]:
!echo "AJ001005.1\nD44590.1\nD44561.1\nD44567.1\nAB983290.1\nHM850200.1\nJQ933298.1\nJQ933366.1\nGU271224.1\nHM850213.1\nHM853899.1" > remove.txt

!cat remove.txt

AJ001005.1
D44590.1
D44561.1
D44567.1
AB983290.1
HM850200.1
JQ933298.1
JQ933366.1
GU271224.1
HM850213.1
HM853899.1


In [6]:
from Bio import SeqIO

ids=[]
temp=[]
blacklist=[]
accessions = open('remove.txt','r')
dropped=0

for line in accessions:
    ids.append(line.strip())
    

recs_to_keep = {'rbcL': []}
recs_to_drop = {'rbcL': ids}

print "Blacklist recs to drop: %s" %recs_to_drop['rbcL']

for key in records.keys():
    r = records[key]
        
    if not r.id in recs_to_drop['rbcL']: 
        recs_to_keep['rbcL'].append(r.id)
        temp.append(r)
    
    else:
        print "exclude: %s" %r.id
        dropped+=1
        blacklist.append(r.id)
        
out = open('rbcL_cleanforSATIVA_eyorks.gb', 'w')
SeqIO.write(temp, out, 'genbank')
out.close()

print "Found blacklist records: %s (of %s)" %(dropped,len(recs_to_drop['rbcL']))
print "Kept: %s" %len(temp)

Blacklist recs to drop: ['AJ001005.1', 'D44590.1', 'D44561.1', 'D44567.1', 'AB983290.1', 'HM850200.1', 'JQ933298.1', 'JQ933366.1', 'GU271224.1', 'HM850213.1', 'HM853899.1']
exclude: D44567.1
exclude: HM850200.1
exclude: D44590.1
exclude: HM850213.1
exclude: JQ933366.1
exclude: JQ933298.1
exclude: AB983290.1
exclude: HM853899.1
exclude: D44561.1
exclude: GU271224.1
Found blacklist records: 10 (of 11)
Kept: 2309


Now re-import the cleaned Genbank file

In [7]:
from Bio import SeqIO

records = SeqIO.to_dict(SeqIO.parse(open('rbcL_cleanforSATIVA_eyorks.gb','r'),'genbank'))

Here is the first Genbank format record:

In [8]:
for r in records.keys():
    print records[r].format('genbank')
    break

LOCUS       AF206823                1377 bp    DNA     linear   PLN 26-JUL-2016
DEFINITION  Stellaria media ribulose 1,5-bisphosphate carboxylase (rbcL) gene,
            partial cds; chloroplast gene for chloroplast product.
ACCESSION   AF206823
VERSION     AF206823.1
KEYWORDS    .
SOURCE      chloroplast Stellaria media
  ORGANISM  Stellaria media
            Eukaryota; Viridiplantae; Streptophyta; Embryophyta; Tracheophyta;
            Spermatophyta; Magnoliophyta; eudicotyledons; Gunneridae;
            Pentapetalae; Caryophyllales; Caryophyllaceae; Alsineae; Stellaria.
REFERENCE   1  (bases 1 to 1377)
  AUTHORS   Soltis,P.S., Soltis,D.E. and Chase,M.W.
  TITLE     Angiosperm phylogeny inferred from multiple genes as a tool for
            comparative biology
  JOURNAL   Unpublished
REFERENCE   2  (bases 1 to 1377)
  AUTHORS   Soltis,P.S., Soltis,D.E. and Chase,M.W.
  TITLE     Direct Submission
  JOURNAL   Submitted (19-NOV-1999) School of Biological Sciences, Washington
         

Iterate over all records and check is taxid is subspecies, if yes, replace with taxid of species and extract taxonomy line. If it's a subspecies record the record ID for further processing.

In [9]:
from collections import defaultdict

taxon_to_taxid = {}
recs_to_adjust = []
taxon_to_recs = defaultdict(list)

for key in records.keys():
    r = records[key]
   
    source = [f for f in r.features if f.type == 'source'][0]
    if (len(source.qualifiers['organism'][0].split(" ")) == 2):
        print source.qualifiers['organism'][0],
        if 'db_xref' in source.qualifiers:
#            print source.qualifiers['db_xref']
            for t in source.qualifiers['db_xref']:
#                print t
                if 'taxon' in t:
                    if not source.qualifiers['organism'][0] in taxon_to_taxid:
                        print " .. add to records"
                        taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                    else:
                        print " .. already covered"
                    taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
    elif (len(source.qualifiers['organism'][0].split(" ")) > 2 and source.qualifiers['organism'][0].split(" ")[1] == 'sp.'):
        print source.qualifiers['organism'][0],
        if 'db_xref' in source.qualifiers:
            for t in source.qualifiers['db_xref']:
                if 'taxon' in t:
                    if not str(t) in taxon_to_taxid:
                        taxon_to_taxid[source.qualifiers['organism'][0]] = str(t)
                    else:
                        print " .. already covered" 
                    taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
    else:
        print "subspecies: %s" %source.qualifiers['organism'][0]
        recs_to_adjust.append(r.id)

Stellaria media  .. add to records
Brassica nigra  .. add to records
Neottia ovata  .. add to records
subspecies: Pinus sylvestris var. mongolica
Lilium pyrenaicum  .. add to records
Juncus tenuis  .. add to records
Brassica oleracea  .. add to records
Gnaphalium uliginosum  .. add to records
Solanum dulcamara  .. add to records
Acer pseudoplatanus  .. add to records
Abies alba  .. add to records
Rumex hydrolapathum  .. add to records
Asplenium ruta-muraria  .. add to records
Sherardia arvensis  .. add to records
Persicaria capitata  .. add to records
subspecies: Dactylorhiza viridis var. viridis
Mercurialis annua  .. add to records
Goodyera repens  .. add to records
Oxalis acetosella  .. add to records
Reynoutria japonica  .. add to records
Sagina maritima  .. add to records
Chaerophyllum temulum  .. add to records
Malva pusilla  .. add to records
Goodyera repens  .. already covered
Parapholis strigosa  .. add to records
Blitum bonus-henricus  .. add to records
Fallopia sachalinensis 

For the records that were identified as being _subspecies_ reduce to species and check whether the taxid for the species had already been encountered. If not we'd need to fetch it from NCBI. I have identified one record which is not dealt with properly here, because it is a hybrid, so I have set up the script to manually remove this record, and two others which are in the Genbank file but not the alignment and so also need removing.

In [10]:
from collections import defaultdict

to_fetch=defaultdict(list)

for key in records.keys():
    r = records[key]
    if r.id in ['U06805.1']:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        print "%s: Manually removed hybrid" %(adjust_from)
    elif r.id in recs_to_adjust:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        adjust_to = " ".join(adjust_from.split(" ")[:2])
        print "%s -> %s" %(adjust_from,adjust_to)
        if adjust_to in taxon_to_taxid:
            taxon_to_recs[adjust_to].append(r.id)
        else:
            to_fetch[adjust_to].append(r.id)

Pinus sylvestris var. mongolica -> Pinus sylvestris
Dactylorhiza viridis var. viridis -> Dactylorhiza viridis
Amaranthus hybridus subsp. cruentus -> Amaranthus hybridus
Solidago canadensis var. canadensis -> Solidago canadensis
Arrhenatherum elatius subsp. elatius -> Arrhenatherum elatius
Prunus domestica subsp. intermedia -> Prunus domestica
Solanum nigrum subsp. nigrum -> Solanum nigrum
Silene latifolia subsp. alba -> Silene latifolia
Hieracium levicaule subsp. lepidulum -> Hieracium levicaule
Cyrtomium fortunei var. clivicola -> Cyrtomium fortunei
Picea abies var. abies -> Picea abies
Athyrium filix-femina var. angustum -> Athyrium filix-femina
Echinochloa crus-galli var. crus-galli -> Echinochloa crus-galli
Thelypteris palustris var. pubescens -> Thelypteris palustris
Solidago canadensis var. scabra -> Solidago canadensis
Populus alba var. alba -> Populus alba
Silene vulgaris subsp. vulgaris -> Silene vulgaris
Asplenium trichomanes subsp. hastatum -> Asplenium trichomanes
Pteris cr

Check if we are good or if any are missing.

In [11]:
if to_fetch:
    print "need to fetch some taxids"
else:
    print "Have taxids for all records"

need to fetch some taxids


In [12]:
print to_fetch

defaultdict(<type 'list'>, {'Bromus diandrus': ['HQ600435.1', 'HM849828.1', 'KF712964.1'], 'Dactylorhiza incarnata': ['KM360748.1'], 'Cotoneaster horizontalis': ['GU363807.1'], 'Origanum vulgare': ['JX880022.1'], 'Odontites vernus': ['FJ395565.1'], 'Matricaria chamomilla': ['JN892268.1'], 'Hieracium levicaule': ['GQ120446.1'], 'Equisetum telmateia': ['AY226135.1'], 'Thymus praecox': ['KM361013.1', 'JN892956.1'], 'Brassica rapa': ['GQ184366.1', 'GQ184364.1', 'GQ184367.1', 'GQ184365.1']})


In [13]:
from Bio import Entrez
Entrez.email = "callumjmacgregor@gmail.com"

for binomial in to_fetch:
    print binomial
    handle = Entrez.esearch(db="Taxonomy", term=binomial)
    record = Entrez.read(handle)
    print record["IdList"][0]
    taxon_to_taxid[binomial] = record["IdList"][0]
    
    taxon_to_recs[binomial] = to_fetch[binomial]

Bromus diandrus
37722
Dactylorhiza incarnata
230574
Cotoneaster horizontalis
690320
Origanum vulgare
39352
Odontites vernus
644195
Matricaria chamomilla
98504
Hieracium levicaule
651083
Equisetum telmateia
3260
Thymus praecox
347388
Brassica rapa
3711


Write taxids to file and fetch full taxonomy for all of them using taxit from the taxtastic package.

In [14]:
taxids = []

out=open("taxids.txt",'w')
for sp in taxon_to_taxid:
    taxids.append(taxon_to_taxid[sp])
    out.write(taxon_to_taxid[sp]+"\n")
out.close()

Create tab-delimited text file with full taxonomic tree for each taxid.

In [15]:
!taxit taxtable -d /usr/bin/taxonomy.db -t taxids.txt -o taxa.csv

In order to make our lives easier downstream we will limit ourselves to only a defined set of the most common taxonomic levels, that should be known for pretty much all taxa: superkingdom, phylum, class, order, family, genus, species.

Extract 'taxonomy string' for a specific set of taxonomic levels.

In [16]:
from collections import defaultdict

tax_levels=['superkingdom','phylum','class','order','family','genus','species']
indices = []
taxdict = defaultdict(list)
taxids_to_taxonomy = {}

infile=open("taxa.csv",'r')
header=infile.next()

header_as_list=header.strip().replace('"','').split(",")
for i in range(len(header_as_list)):
#    print header_as_list[i]
    if header_as_list[i] in tax_levels:
#        print "\t"+header_as_list[i],i
        indices.append(i)

for line in infile:
    line_as_list=line.strip().replace('"',"").split(",")
    taxdict[line_as_list[0]] = line_as_list[1:]

infile.close()

for t in taxids:
    print t,
#    print taxdict[t]
    taxonomy=""
    for i in range(len(tax_levels)):
        if taxdict[t][indices[i]-1] == "":
            taxonomy+='unknown'+';'
        else:
            taxonomy+=taxdict[taxdict[t][indices[i]-1]][2]+";"
    print taxonomy[:-1]
    taxids_to_taxonomy[t] = taxonomy[:-1]


335201 Eukaryota;Streptophyta;Liliopsida;Poales;Poaceae;Deschampsia;Deschampsia setacea
174663 Eukaryota;Streptophyta;unknown;Ericales;Polemoniaceae;Polemonium;Polemonium caeruleum
207839 Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
53809 Eukaryota;Streptophyta;unknown;Oxalidales;Oxalidaceae;Oxalis;Oxalis pes-caprae
262952 Eukaryota;Streptophyta;unknown;Polypodiales;Pteridaceae;Pteris;Pteris tremula
115624 Eukaryota;Streptophyta;unknown;Caryophyllales;Caryophyllaceae;Herniaria;Herniaria glabra
55378 Eukaryota;Streptophyta;unknown;Ericales;Balsaminaceae;Impatiens;Impatiens capensis
231488 Eukaryota;Streptophyta;Liliopsida;Liliales;Melanthiaceae;Paris;Paris quadrifolia
29760 Eukaryota;Streptophyta;unknown;Vitales;Vitaceae;Vitis;Vitis vinifera
234465 Eukaryota;Streptophyta;Liliopsida;Poales;Cyperaceae;Carex;Carex remota
39358 Eukaryota;Streptophyta;unknown;Lamiales;Lamiaceae;Prunella;Prunella vulgaris
3854 Eukaryota;Streptophyta;unknown;Fabales;

Write out the *.tax file for SATIVA.

In [17]:
out=open("tax_for_SATIVA.tax", 'w')

for sp in taxon_to_recs:
    for rec in taxon_to_recs[sp]:
        out.write("%s\t%s\n" %(rec,taxids_to_taxonomy[taxon_to_taxid[sp]]))
        
out.close()

Check it

In [18]:
!head tax_for_SATIVA.tax

JN892158.1	Eukaryota;Streptophyta;Liliopsida;Poales;Poaceae;Deschampsia;Deschampsia setacea
KF997406.1	Eukaryota;Streptophyta;unknown;Ericales;Polemoniaceae;Polemonium;Polemonium caeruleum
EF394237.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
KC795814.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
KC795822.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AB575106.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AB598690.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AB598689.1	Eukaryota;Streptophyta;unknown;Polypodiales;Dryopteridaceae;Cyrtomium;Cyrtomium fortunei
AM235044.1	Eukaryota;Streptophyta;unknown;Oxalidales;Oxalidaceae;Oxalis;Oxalis pes-caprae
JQ412403.1	Eukaryota;Streptophyta;unknown;Oxalidales;Oxalidaceae;Oxalis;Oxalis pes-caprae


Let's now run SATIVA. We'll first need to install it to this location.

In [19]:
!git clone --recursive https://github.com/amkozlov/sativa.git

fatal: destination path 'sativa' already exists and is not an empty directory.


In [20]:
cd sativa/

/home/working/AgainstRefDB/Curat1/FINAL/Sativa/sativa


Record SHA-1 checksum for the current commit for reproducibility:

In [21]:
!git log -1 | head -n 1

commit 8a99328f3f5382f7f541526878d049415af70999


In [22]:
!./install.sh

Your compiler: gcc 4.8
Building AVX: yes
Building AVX2: yes
make: Entering directory `/home/working/AgainstRefDB/Curat1/FINAL/Sativa/sativa/raxml'
rm -rf builddir.*
rm -f unpack.*.stamp
make: Leaving directory `/home/working/AgainstRefDB/Curat1/FINAL/Sativa/sativa/raxml'

Done!


In [23]:
cd ..

/home/working/AgainstRefDB/Curat1/FINAL/Sativa


In [24]:
!./sativa/sativa.py -s alignment.phy -t tax_for_SATIVA.tax -x zoo -n rbcL -o ./ -T 5 -v


SATIVA v0.9-55-g0cbb090, released on 2016-06-28. Last version: https://github.com/amkozlov/sativa 
By A.Kozlov and J.Zhang, the Exelixis Lab. Based on RAxML 8.2.3 by A.Stamatakis.

SATIVA was called as follows:

./sativa/sativa.py -s alignment.phy -t tax_for_SATIVA.tax -x zoo -n rbcL -o ./ -T 5 -v

Mislabels search is running with the following parameters:
 Alignment:                        alignment.phy
 Taxonomy:                         tax_for_SATIVA.tax
 Output directory:                 /home/working/AgainstRefDB/Curat1/FINAL/Sativa
 Job name / output files prefix:   rbcL
 Model of rate heterogeneity:      AUTO
 Confidence cut-off:               0.000000
 Number of threads:                5

*** STEP 1: Building the reference tree using provided alignment and taxonomic annotations ***

=> Loading taxonomy from file: tax_for_SATIVA.tax ...

==> Loading reference alignment from file: alignment.phy ...

Guessing input format: not fasta
Guessing input format: not phylip_relaxed
===> 

#SATIVA post-processing

We now want to exclude records SATIVA doesn't like, and redo the alignment and tree-building stages.

In [25]:
from reprophylo import *

In [27]:
pj=Project('target_locus_eyorks.csv', pickle='post_SATIVA_final.pkl', git=False)

DEBUG:Cloud:Log file (/root/.picloud/cloud.log) opened


In [28]:
pj.read_embl_genbank(['rbcL_nr_pre_Sativa_eyorks.gb'])

/home/reprophylo/reprophylo.py:1015: UserWarning: Version control off
  warnings.warn('Version control off')


In [29]:
pj.extract_by_locus()

Set the stage:

In [30]:
recs_to_keep = {'rbcL': []}
recs_to_drop = {'rbcL': []}

print "to drop: %s" %len(recs_to_drop['rbcL'])
print "to keep: %s" %len(recs_to_keep['rbcL'])

print "TOTAL: %s" %(len(recs_to_drop['rbcL']) + len(recs_to_keep['rbcL']))

to drop: 0
to keep: 0
TOTAL: 0


Record all record IDs and preliminarily assign them the status 'keep'

In [31]:
for r in pj.records_by_locus['rbcL']:
    recs_to_keep['rbcL'].append("_".join(r.id.split("_")[:-1]))

In [32]:
print "to drop: %s" %len(recs_to_drop['rbcL'])
print "to keep: %s" %len(recs_to_keep['rbcL'])

print "TOTAL: %s" %(len(recs_to_drop['rbcL']) + len(recs_to_keep['rbcL']))

to drop: 0
to keep: 2319
TOTAL: 2319


In [33]:
rec_mislabelled = {}
sativa=open('rbcL.mis', 'r')

for line in sativa:
    if not line.startswith(";"):
        rec_mislabelled[line.split("\t")[0]] = line.split("\t")[4]

In [34]:
print len(rec_mislabelled)

162


Adjust lists to move these sequences to the 'recs_to_drop' list

In [35]:
print "Initial keeplist length: %s" %len(recs_to_keep['rbcL'])
added=0
removed=0
for r in rec_mislabelled:
    print r
    if not r in recs_to_drop['rbcL']:
        print "\tadd to droplist"
        recs_to_drop['rbcL'].append(r)
        added+=1
    for i in reversed(range(len(recs_to_keep['rbcL']))):
        if r in recs_to_keep['rbcL'][i]:
            print "\tremove from keeplist"
            del(recs_to_keep['rbcL'][i])
            removed+=1
            break
            
print "Added to droplist: %s" %added
print "Removed from keeplist: %s" %removed
print "Remaining in keeplist: %s" %len(recs_to_keep['rbcL'])

Initial keeplist length: 2319
KM361012.1
	add to droplist
	remove from keeplist
KJ841324.1
	add to droplist
	remove from keeplist
JQ230997.1
	add to droplist
	remove from keeplist
HM850026.1
	add to droplist
	remove from keeplist
JQ412374.1
	add to droplist
	remove from keeplist
KP098266.1
	add to droplist
	remove from keeplist
JN891551.1
	add to droplist
	remove from keeplist
KT627040.1
	add to droplist
	remove from keeplist
JN892956.1
	add to droplist
	remove from keeplist
JN893464.1
	add to droplist
	remove from keeplist
JN891378.1
	add to droplist
	remove from keeplist
HM849876.1
	add to droplist
	remove from keeplist
HQ590234.1
	add to droplist
	remove from keeplist
JN891008.1
	add to droplist
	remove from keeplist
KF613040.1
	add to droplist
	remove from keeplist
JF265398.1
	add to droplist
	remove from keeplist
KF186509.1
	add to droplist
	remove from keeplist
JF832056.1
	add to droplist
	remove from keeplist
KJ916496.1
	add to droplist
	remove from keeplist
HM850091.1
	add to d

Check the numbers at the end of the list to confirm everything's worked:

In [36]:
print "to drop: %s" %len(recs_to_drop['rbcL'])
print "to keep: %s" %len(recs_to_keep['rbcL'])

print "TOTAL: %s" %(len(recs_to_drop['rbcL']) + len(recs_to_keep['rbcL']))

to drop: 162
to keep: 2157
TOTAL: 2319


Write out good records only in full Genbank format including all metadata. Adjust to valid species and taxids before doing so.

Identify records with subspecies.

In [37]:
from collections import defaultdict

Seqs = []
taxon_to_taxid = {}
recs_to_adjust = defaultdict(list)
taxon_to_recs = defaultdict(list)

for r in pj.records:
    if r.id in recs_to_keep['rbcL']:
        Seqs.append(r)
        
        source = [f for f in r.features if f.type == 'source'][0]
        
        #screen for subspecies and record taxids of valid species
        if (len(source.qualifiers['organism'][0].split(" ")) == 2):
            print source.qualifiers['organism'][0],
            if 'db_xref' in source.qualifiers:
                for t in source.qualifiers['db_xref']:
                    if 'taxon' in t:
                        if not source.qualifiers['organism'][0] in taxon_to_taxid:
                            print " .. add to records"
                            taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                        else:
                            print " .. already covered"
                        taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
            elif (len(source.qualifiers['organism'][0].split(" ")) > 2 and source.qualifiers['organism'][0].split(" ")[1] == 'sp.'):
                print source.qualifiers['organism'][0],
                if 'db_xref' in source.qualifiers:
                    for t in source.qualifiers['db_xref']:
                        if 'taxon' in t:
                            if not source.qualifiers['organism'][0] in taxon_to_taxid:
                                print " .. add to records"
                                taxon_to_taxid[source.qualifiers['organism'][0]] = t.split(":")[1]
                            else:
                                print " .. already covered"
                            taxon_to_recs[source.qualifiers['organism'][0]].append(r.id)
            else:
                print "subspecies: %s" %source.qualifiers['organism'][0]
                recs_to_adjust[r.id] = []

Carpinus betulus  .. add to records
Solanum dulcamara  .. add to records
Pteris vittata  .. add to records
Veronica anagallis-aquatica  .. add to records
Epipactis helleborine  .. add to records
Fallopia baldschuanica  .. add to records
Hypericum perforatum  .. add to records
Goodyera repens  .. add to records
Galium odoratum  .. add to records
Sinapis arvensis  .. add to records
Borago officinalis  .. add to records
Lonicera periclymenum  .. add to records
Sherardia arvensis  .. add to records
Triticum aestivum  .. add to records
Galium palustre  .. add to records
Syringa vulgaris  .. add to records
Solanum tuberosum  .. add to records
Geranium robertianum  .. add to records
Calystegia silvatica  .. add to records
Goodyera repens  .. already covered
Fallopia sachalinensis  .. add to records
Blysmus compressus  .. add to records
Quercus robur  .. add to records
Poa trivialis  .. add to records
Caltha palustris  .. add to records
Fritillaria meleagris  .. add to records
Brachypodium pin

Check if any of the taxa that were considered dubious are not already covered in the list after removing subspecies.

In [38]:
from collections import defaultdict

to_fetch = defaultdict(list)

for r in Seqs:
    if r.id in recs_to_adjust:
        source = [f for f in r.features if f.type == 'source'][0]
        adjust_from = source.qualifiers['organism'][0]
        adjust_to = " ".join(adjust_from.split(" ")[:2])
        print "%s -> %s" %(adjust_from,adjust_to)
        if adjust_to in taxon_to_taxid:
            taxon_to_recs[adjust_to].append(r.id)
            recs_to_adjust[r.id].append(adjust_to)
            recs_to_adjust[r.id].append(taxon_to_taxid[adjust_to])
        else:
            to_fetch[adjust_to].append(r.id)
            recs_to_adjust[r.id].append(adjust_to)

In [39]:
print len(to_fetch)

0


In [40]:
for r in Seqs:
    if r.id in recs_to_adjust:
        print "adjusting %s" %r.id
        source = r.features[0]
        print "%s\t->\t%s" %(source.qualifiers['organism'][0],recs_to_adjust[r.id][0])
        source = r.features[0]
        source.qualifiers['organism'] = [recs_to_adjust[r.id][0]]
        for i in range(len(source.qualifiers['db_xref'])):
            if 'taxon' in source.qualifiers['db_xref'][i]:
                source.qualifiers['db_xref'][i] = "taxon:"+recs_to_adjust[r.id][1]
            if not 'note' in source.qualifiers:
                source.qualifiers['note'] = ['modified from original record: adjusted to species']
            else:
                source.qualifiers['note'].append('modified from original record: adjusted to species')

Write out only good records with all metadata in Genbank format

In [41]:
from Bio import SeqIO

out = open('eyorks_flora_final.gb','w')
SeqIO.write(Seqs, out, "genbank")
out.close()

#Prepare ref db for pplacer

Exclude the records that were highlighted by SATIVA. Then realign and rebuild phylogeny.

In [42]:
pj.exclude(start_from_max=False, **recs_to_drop)

In [43]:
species=[]
for r in pj.records:
    if r.id in recs_to_keep['rbcL']:
        source = [f for f in r.features if f.type == 'source'][0]
        species.append(" ".join(source.qualifiers['organism'][0].split()[0:2]))
        
total=len(species)
species=list(set(species))

print "Total number of records: %i" %total
print "Number of species (see below for full list): %i" %len(species)
print sorted(species)

Total number of records: 2157
Number of species (see below for full list): 1034
['Abies alba', 'Abies concolor', 'Abies firma', 'Abies grandis', 'Abies homolepis', 'Abies lasiocarpa', 'Abies nordmanniana', 'Abies procera', 'Abies veitchii', 'Acaena novae-zelandiae', 'Acer campestre', 'Acer platanoides', 'Acer pseudoplatanus', 'Achillea millefolium', 'Aconitum napellus', 'Acorus calamus', 'Actaea spicata', 'Adonis annua', 'Adoxa moschatellina', 'Aegopodium podagraria', 'Aesculus hippocastanum', 'Aethusa cynapium', 'Agrimonia eupatoria', 'Agrimonia procera', 'Agrostis canina', 'Agrostis capillaris', 'Agrostis gigantea', 'Agrostis scabra', 'Agrostis stolonifera', 'Aira caryophyllea', 'Aira praecox', 'Ajuga reptans', 'Alcea rosea', 'Alchemilla glabra', 'Alchemilla mollis', 'Alchemilla xanthochlora', 'Alisma lanceolatum', 'Alisma plantago-aquatica', 'Alliaria petiolata', 'Allium carinatum', 'Allium oleraceum', 'Allium scorodoprasum', 'Allium triquetrum', 'Allium ursinum', 'Allium vineale', 

Configure alignment

Again we first need to remove a few records without protein sequences.

In [44]:
from Bio import SeqFeature

strands=[]
notranslation=[]
nogene=[]

remove=[]

for r in pj.records:
    
    print "\nprocessing record: '%s' " %r.id,
    if r.features:
        for feature in r.features:
            if feature.type == 'CDS':
                qualifiers = feature.qualifiers
                if 'translation' not in qualifiers:
                    print "No translation"
                    notranslation.append(r.id)
                    remove.append(r.id)
                elif 'gene' not in qualifiers:
                    print "No gene"
                    nogene.append(r.id)
                    remove.append(r.id)
                    
                    
print "\nNumber without translations: %s" %len(notranslation)
print "Bad records: %s" %notranslation

print "Number without gene: %s" %len(nogene)
print "Bad records: %s" %nogene

out = open('remove_post.txt','w')

for item in remove:
    out.write("%s\n" %item)
    
out.close()


processing record: 'JN893788.1'  
processing record: 'KJ841577.1'  
processing record: 'HM559497.1'  
processing record: 'AY034021.1'  
processing record: 'KF724370.1'  
processing record: 'HE963482.1'  
processing record: 'KT695421.1'  
processing record: 'JN005477.1'  
processing record: 'JN893409.1'  
processing record: 'KJ841573.1'  
processing record: 'KT740813.1'  
processing record: 'KJ204369.1'  
processing record: 'HQ644069.1'  
processing record: 'AY328025.1'  
processing record: 'KJ841341.1'  
processing record: 'DQ673303.1'  
processing record: 'HF572814.1'  
processing record: 'KP963378.1'  
processing record: 'KF997298.1'  
processing record: 'KF163549.1'  
processing record: 'FJ571330.1'  
processing record: 'KJ863243.1'  
processing record: 'AM999788.1'  
processing record: 'KX163018.1'  
processing record: 'KX513086.1'  
processing record: 'KT695415.1'  
processing record: 'AY395537.1'  
processing record: 'AM849347.1'  
processing record: 'KJ593671.1'  
processing re

In [45]:
!cat remove_post.txt

D44590.1
D44561.1
D44567.1


In [46]:
from Bio import SeqIO

ids=[]
temp=[]
blacklist=[]
accessions = open('remove_post.txt','r')
dropped=0

for line in accessions:
    ids.append(line.strip())
    
    
recs_to_keep = {'rbcL': []}
recs_to_drop = {'rbcL': ids}

print "Blacklist recs to drop: %s" %recs_to_drop['rbcL']

for r in pj.records:
    
    if not r.id in recs_to_drop['rbcL']:
        recs_to_keep['rbcL'].append(r.id)
        temp.append(r)
        
    else:
        print "exclude: %s" %r.id
        dropped+=1
        blacklist.append(r.id)
        
out = open('postSATIVA_cleaned.gb','w')
SeqIO.write(temp, out, 'genbank')
out.close()

out = open('postSATIVA_cleaned.fa','w')
SeqIO.write(temp, out, 'fasta')
out.close()

print "Found blacklist records: %s (of %s)" %(dropped,len(recs_to_drop['rbcL']))
print "Kept: %s" %len(temp)

Blacklist recs to drop: ['D44590.1', 'D44561.1', 'D44567.1']
exclude: D44590.1
exclude: D44561.1
exclude: D44567.1
Found blacklist records: 3 (of 3)
Kept: 2316


In [47]:
pickle_pj(pj, 'post_SATIVA_final.pkl')

'post_SATIVA_final.pkl'

In [48]:
pj = Project('target_locus_eyorks.csv', pickle='post_SATIVA_align.pkl', git=False)

In [49]:
pj.read_embl_genbank(['postSATIVA_cleaned.gb'])

In [50]:
pj.extract_by_locus()

In [51]:
mafft_linsi = AlnConf(pj,
                     method_name='mafftLinsi-SATIVA',
                     CDSAlign=True,
                     codontable=11,
                     program_name='mafft',
                     cmd='mafft',
                     loci=['rbcL'],
                     cline_args={'localpair': True,
                                'maxiterate': 1000}
                     )

mafft --localpair --maxiterate 1000 887121477660388.49_CDS_proteins_rbcL.fasta


Perform alignment

In [52]:
pj.align([mafft_linsi])

In [54]:
print pj.used_methods['mafftLinsi-SATIVA']

AlnConf named mafftLinsi-SATIVA with ID 887121477660388.49
Loci: rbcL 
Created on: Fri Oct 28 13:13:08 2016
Commands:
rbcL: mafft --localpair --maxiterate 1000 887121477660388.49_CDS_proteins_rbcL.fasta

Environment:
Platform: Linux-4.4.0-45-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.68
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.3
 User: BioPC5
 Program and version: MAFFT v7.123b\nPal2Nal v14
 Program reference:Katoh
 Standley 2013 (Molecular Biology and Evolution 30:772-780) MAFFT multiple sequence alignment software version 7: improvements in performance and usability.\nMikita Suyama
 David Torrents
 and Peer Bork (2006) PAL2NAL: robust conversion of protein sequence alignments into the corresponding codon alignments.Nucleic Acids Res. 34
 W609-W612.
execution time:
3628.4

In [55]:
pickle_pj(pj, 'post_SATIVA_align.pkl')

'post_SATIVA_align.pkl'

Re-clip the new alignment

In [56]:
pj.alignments

{'rbcL@mafftLinsi-SATIVA': <<class 'Bio.Align.MultipleSeqAlignment'> instance (2316 records of length 1551, IUPACAmbiguousDNA()) at 7fcfc60c99d0>}

In [57]:
gappyout = TrimalConf(pj,
                     method_name='gappyout',
                     program_name='trimal',
                     cmd='default',
                     alns=['rbcL@mafftLinsi-SATIVA'],
                     trimal_commands={'gappyout': True}
                     )

trimal -in 130381477664600.04_rbcL@mafftLinsi-SATIVA.fasta -gappyout


In [58]:
pj.trim([gappyout])

Alignment rbcL@mafftLinsi-SATIVA@gappyout has undetermined sequences (1 bp or less) which will be dropped: ['GU271224.1_f0', 'HM850213.1_f0', 'JQ933366.1_f0', 'JQ933298.1_f0', 'HM853899.1_f0', 'HM850200.1_f0', 'AB983290.1_f0']


In [59]:
print pj.used_methods['gappyout']

TrimalConf named gappyout with ID 130381477664600.04
Alignments: rbcL@mafftLinsi-SATIVA 
Created on: Fri Oct 28 14:23:20 2016
Commands:
rbcL@mafftLinsi-SATIVA@gappyout: trimal -in 130381477664600.04_rbcL@mafftLinsi-SATIVA.fasta -gappyout

Environment:Platform: Linux-4.4.0-45-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.68
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.3
 User: BioPC5
 Program and version: trimAl 1.2rev59
 Program reference: Salvador Capella-Gutierrez; Jose M. Silla-Martinez; Toni Gabaldon. trimAl: a tool for automated alignment trimming in large-scale phylogenetic analyses. Bioinformatics 2009 25: 1972-1973.
execution time:
10.9124538898

Core Methods section sentence:
The alignment(s) rbcL@mafftLinsi-SATIVA were trimmed using the program trimAl 1.2rev59 [1].

Ref

In [60]:
pj.trimmed_alignments

{'rbcL@mafftLinsi-SATIVA@gappyout': <<class 'Bio.Align.MultipleSeqAlignment'> instance (2309 records of length 174, IUPACAmbiguousDNA()) at 7fcfc497b4d0>}

In [61]:
files = pj.write_trimmed_alns(id=['record_id','source_organism'],format='fasta')
files

['rbcL@mafftLinsi-SATIVA@gappyout_trimmed_aln.fasta']

In [62]:
from Bio import AlignIO

clipped_aln = pj.trimmed_alignments["rbcL@mafftLinsi-SATIVA@gappyout"]

Write out the alignment for use by pplacer. Need to modify the record id, removing the _f0 that is appended by ReproPhylo.

In [64]:
from Bio import AlignIO

aln = pj.fta('rbcL@mafftLinsi-SATIVA@gappyout')

for r in aln:
        r.id = "_".join(r.id.split("_")[:-1])
        r.description = r.id
        
out=open('../rbcL_eyorks_clean_alignment.fasta', 'w')
AlignIO.write(aln, out, 'fasta')
out.close()

out=open('../rbcL_eyorks_clean_alignment.phylip', 'w')
AlignIO.write(aln, out, 'phylip-relaxed')
out.close()

returning trimmed alignment object rbcL@mafftLinsi-SATIVA@gappyout


With everything else finished, we can try to construct a final tree:

In [68]:
raxml = RaxmlConf(pj,
                 method_name='raxml-mafftLinsi-SATIVA-clipped',
                 program_name='raxmlHPC-PTHREADS-SSE3',
                 keepfiles=True,
                 cmd='raxmlHPC-PTHREADS-SSE3',
                 preset='fa',
                 alns=['rbcL@mafftLinsi-SATIVA@gappyout'],
                 model='GAMMA',
                 threads=5,
                 cline_args={'-#':100})

raxmlHPC-PTHREADS-SSE3 -f a -m GTRGAMMA -n 743581477665391.27_rbcL@mafftLinsi-SATIVA@gappyout0 -p 825 -s 743581477665391.27_rbcL@mafftLinsi-SATIVA@gappyout.fasta -T 5 -x 462 -N 100


In [69]:
pj.tree([raxml])

In [70]:
print pj.used_methods['raxml-mafftLinsi-SATIVA-clipped']

RaxmlConf named raxml-mafftLinsi-SATIVA-clipped with ID 743581477665391.27
Alignments: rbcL@mafftLinsi-SATIVA@gappyout 
Created on: Fri Oct 28 14:36:31 2016
Commands:
rbcL@mafftLinsi-SATIVA@gappyout:
raxmlHPC-PTHREADS-SSE3 -f a -m GTRGAMMA -n 743581477665391.27_rbcL@mafftLinsi-SATIVA@gappyout0 -p 825 -s 743581477665391.27_rbcL@mafftLinsi-SATIVA@gappyout.fasta -T 5 -x 462 -N 100

Environment:
Platform: Linux-4.4.0-45-generic-x86_64-with-Ubuntu-14.04-trusty
 Processor: x86_64
 Python build: defaultMar 22 2014 22:59:56
 Python compiler: GCC 4.8.2
 Python implementation: CPython
 Python version: 2.7.6
 ete2 version: 2.2.1072
 biopython version: 1.68
 dendropy version: 3.12.2
 cloud version: 2.8.5
 reprophylo version 1.3
 User: BioPC5
 Program and version: raxmlHPC-PTHREADS-SSE3RAxML version 8.0.12
 Program reference: A. Stamatakis: RAxML Version 8: A tool for Phylogenetic Analysis and Post-Analysis of Large Phylogenies. In Bioinformatics
 2014.
execution time:
7882.38629913

Core Methods s

In [71]:
pickle_pj(pj, 'post_SATIVA_align.pkl')

'post_SATIVA_align.pkl'

Annotate final tree.

In [72]:
pj.add_qualifier_from_source('organism')

In [73]:
supports = {'black': [100,95],
           'dimgray': [95,75],
           'silver': [75,50]}

In [74]:
for species in ['Ginkgo_biloba']:
    pj.if_this_then_that(species, 'organism', 'outgroup', 'outgroup',
                        mode='part')

In [75]:
pj.clear_tree_annotations()
pj.annotate('.', 'outgroup', 'outgroup',
           ['source_organism','record_id'],
           node_support_dict=supports, multifurc=None,
           html='mafftLinsi-clipped-raxml-post_SATIVA.html',
           fsize=13)

In [76]:
pickle_pj(pj, 'post_SATIVA_align.pkl')

'post_SATIVA_align.pkl'

Write out final tree in simpler format

In [77]:
%%bash

sed 's/_f[0-9]//g' RAxML_bestTree.743581477665391.27_rbcL@mafftLinsi-SATIVA@gappyout0 > RAxML_bestTree.rbcL@mafftLinsi-SATIVA@gappyout0

In [78]:
%%bash

last_png=$(ls -hlrt *rbcL@mafftLinsi-SATIVA@gappyout.png | tail -n 1 | perl -ne 'chomp; @a=split(" "); print "$a[-1]"')
cp $last_png rbcL_mafftLinsi-SATIVA_clipped_raxml_SATIVA.png

This stage is finished; our final reference database has been curated and the relevant tree files have been produced to proceed with phylogenetic placement.